In [2]:
import numpy as np
import pandas as pd


In [5]:
X = pd.read_parquet("/home/cara/Documents/reddit_analyses/thread-size/Test_outputs/conspiracy_train_X.parquet")
y = pd.read_parquet("/home/cara/Documents/reddit_analyses/thread-size/Test_outputs/conspiracy_train_Y.parquet")['log_thread_size']

In [6]:
CLASS_BIN_EDGES = {
    3: [0.5],
    4: [1 / 3, 2 / 3],
}

In [8]:
classes = 4

In [10]:
bin_edges = [np.log(1) - 1e-3]
bin_edges.append(np.log(2) - 1e-3)
bin_edges.extend(
    [y[y > np.log(1)].quantile(x) for x in CLASS_BIN_EDGES[classes]]
)
bin_edges.append(y.max() + 1e-3)

In [12]:
np.exp(bin_edges)

array([  0.9990005 ,   1.998001  ,   7.        ,  22.        ,
       927.92746365])

In [78]:
y_bins = pd.cut(y, bins=bin_edges, labels=False, include_lowest=True)

In [82]:
thread_classes = {}

for i in range(1, 27):
    thread_classes[i] = y_bins[y == np.log(i)].unique()[0]

df = pd.DataFrame.from_dict(thread_classes, orient='index', columns=["class_name"])
df.index.name = "thread_size"
display(df)

,class_name
thread_size,
1,0
2,1
3,1
4,1
5,1
6,1
7,1
8,2
9,2


In [80]:
df.class_name.unique()

array([0, 1, 2, 3])

In [83]:
for i in df.class_name.unique():
    thread_size_vals = df[df.class_name == i].index
    print(f"Class {i}: [{thread_size_vals.min()}, {thread_size_vals.max()}]")

Class 0: [1, 1]
Class 1: [2, 7]
Class 2: [8, 22]
Class 3: [23, 26]


In [108]:
y_bin_counts = y_bins.value_counts().sort_index()
y_bin_counts.index.name = "Class"
thread_size_bins = [round(np.exp(x)) for x in bin_edges]
i = 0
bin_ranges = [[thread_size_bins[i], thread_size_bins[i]]]
for i in range(1, len(thread_size_bins)-1):
    to_append = [bin_ranges[i-1][1]+1, thread_size_bins[i+1]]
    bin_ranges.append(to_append)
bin_count_df = pd.DataFrame(
    {"Range": bin_ranges, "Threads": y_bin_counts}
)

In [109]:
bin_count_df

,Range,Threads
Class,,
0,"[1, 1]",115
1,"[2, 7]",241
2,"[8, 22]",218
3,"[23, 928]",226


In [3]:
out_dir = "/home/cara/Documents/reddit_analyses/thread-size/Test_outputs"
for subreddit in ['politics', 'crypto']:
    root_dir = f"/home/cara/Documents/reddit_analyses/thread-size/Outputs/Preprocessing/{subreddit}"
    for filename in [f"{subreddit}_test_X.parquet", f"{subreddit}_test_Y.parquet", f"{subreddit}_train_X.parquet", f"{subreddit}_train_Y.parquet"]:
        df = pd.read_parquet(f"{root_dir}/{filename}").head(500)
        df.to_parquet(f"{out_dir}/{filename}")


In [1]:
import pandas as pd
import numpy as np
import joblib

In [13]:
cm_dict = joblib.load("/home/cara/Documents/reddit_analyses/thread-size/Test_outputs/1_Thread_Start/politics/4_model/model_1/model_data/test_confusion_matrix_data.jl")

In [14]:
def get_correct_classes(cm):
    class_correct = []
    for i in range(0,len(cm)):
        class_correct.append(cm[i,i]/sum(cm[i,:]))
    return class_correct

def get_true_class_counts(cm):
    class_counts = []
    for i in range(0, len(cm)):
        class_counts.append(sum(cm[i,:]))
    return class_counts

def get_predicted_class_counts(cm):
    class_counts = []
    for i in range(0, len(cm)):
        class_counts.append(sum(cm[:,i]))
    return class_counts

def get_predicted_class_ratios_df(cm_dict):
    pred_class_ratios = {}
    for k, cm in cm_dict.items():
        pred_class_ratios[k] = get_predicted_class_counts(cm)/cm_dict['CM'].sum()
    true_class_ratios = get_true_class_counts(cm_dict['CM'])/cm_dict['CM'].sum()
    return pred_class_ratios, true_class_ratios
    

In [15]:
pred_class_ratios, true_class_ratios = get_predicted_class_ratios_df(cm_dict)

In [16]:
pred_class_ratios

{'lower': array([0.37275, 0.4996 ]),
 'upper': array([0.49715, 0.62385]),
 'mean': array([0.4345, 0.5655]),
 'std': array([0.03795131, 0.03648622]),
 'CM': array([0.43, 0.57])}

In [17]:
def get_predicted_class_ratios_df(cm_dict):
    pred_class_ratios = {}
    for k, cm in cm_dict.items():
        pred_class_ratios[k] = get_predicted_class_counts(cm)/cm_dict['CM'].sum()
    true_class_ratios = get_true_class_counts(cm_dict['CM'])/cm_dict['CM'].sum()
    df = pd.concat([pd.DataFrame(pred_class_ratios),pd.DataFrame(true_class_ratios)], axis=1).rename(columns={
        "CM": "predicted",
        0: "true"
    })
    return df

In [18]:
get_predicted_class_ratios_df(cm_dict)

,lower,upper,mean,std,predicted,true
0,0.37275,0.49715,0.4345,0.037951,0.43,0.278
1,0.49960,0.62385,0.5655,0.036486,0.57,0.722
